In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_curve



# Read Data

In [15]:
#Import Censored Data
censored_df=pd.read_csv('censored_tweets.csv')
censored_df = pd.DataFrame(censored_df)
censored_df.insert(loc=3, column='y', value=1)
censored_df = censored_df.drop(['id','lang'],axis=1)

#import regular tweets
uncensored_df=pd.read_csv('regular_tweets.csv')
uncensored_df = pd.DataFrame(uncensored_df)
uncensored_df.insert(loc=3, column='y', value=0)
uncensored_df = uncensored_df.drop(['id','lang'],axis=1)

#process tweets
uncensored_df["text"].apply(lambda t: t.lower().translate(str.maketrans('', '', string.punctuation)))
censored_df["text"].apply(lambda t: t.lower().translate(str.maketrans('', '', string.punctuation)))

#merge and shufflem
df = pd.concat([censored_df, uncensored_df], ignore_index=True, sort=False)
df = shuffle(df)
df_x=df["text"]
df_y=df["y"]

In [16]:
df

,text,y
1060,MUSLIMS... just say it. https://t.co/uRcMtOdWGN,1
663,'Master negotiator/dealmaker' @KellyannePolls...,1
2376,That’s what happens when you bring back a show...,0
1396,respect woman https://t.co/4yns5wWs5E,1
3457,Hillary Clinton says Democrats are so socialis...,0
...,...,...
2902,Beautiful https://t.co/LiHZPH2gNG,0
2867,Sneakerhead 🌸✨One of the illustrations I did f...,0
706,ALERT! I NEED YOUR HELP - This is SHOCKING! ...,1
1821,@YasinKesen @cevheriguven @jack Dear @Jack My ...,1


In [17]:
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=4)
print("fraction of censored tweets", y_train.mean())

fraction of censored tweets 0.4909967845659164


In [18]:
x_train

1829    Austria Takes Over Rotating EU Presidency, Ple...
3236    dear internationals,  have so much fun like wh...
2001    How much have the Trump brothers' overseas bus...
528     #Krauthammer is a traitor. Don’t feel bad for ...
3291    this k-army showed her friends how loved and s...
                              ...                        
3693    guess im a vegan now 👀👅😍🔥 https://t.co/kqgeLnl3cE
1237    #Turkish #government #fascism at work: https:/...
3296    Oh and ANOTHER THING.   Dancehall is not Regga...
1739    Military is a family affair of true AMERICAN P...
500     "Immigration is not a Jewish conspiracy, you f...
Name: text, Length: 3110, dtype: object

# Create Spam Bag of Words and Non-Spam Bag of Words

In [19]:

#get bag of censored words & bag of uncensored words
x_train_censored = x_train[y_train == 1]
x_train_uncensored = x_train[y_train == 0]

train_censored_bag = ' '.join(x_train_censored).split(' ')
train_uncensored_bag = ' '.join(x_train_uncensored).split(' ')
common_words = set(train_censored_bag).intersection(set(train_uncensored_bag))

In [20]:
train_censored_bow = dict()
for w in common_words:
    train_censored_bow[w] = train_censored_bag.count(w) / len(train_censored_bag)

In [21]:
train_uncensored_bow = dict()
for w in common_words:
    train_uncensored_bow[w] = train_uncensored_bag.count(w) / len(train_uncensored_bag)

# Predict on Test Set

In [22]:
def predict_text(t, verbose=False):
    #if some word doesnt appear in either spam or non-spam BOW, disregard it
    valid_words = [w for w in t if w in train_censored_bow]
    
    #get the probabilities of each valid word showing up in spam and non-spam BOW
    censored_probs = [train_censored_bow[w] for w in valid_words]
    uncensored_probs = [train_uncensored_bow[w] for w in valid_words]
    
    #print probs if requested
    if verbose:
        data_df = pd.DataFrame()
        data_df['word'] = valid_words
        data_df['censored_prob'] = censored_probs
        data_df['uncensored_prob'] = uncensored_probs
        data_df['ratio'] = [c/u if u > 0 else np.inf for c,u in zip(censored_probs, uncensored_probs)]
        print(data_df)
     
    #calculate spam score as sum of logs for all probabilities
    censored_score = sum([np.log(p) for p in censored_probs]) + np.log(y_train.mean())
    
    #calculate non-spam score as sum of logs for all probabilities
    uncensored_score = sum([np.log(p) for p in uncensored_probs]) + np.log(1-y_train.mean())
    
    #if verbose, report the two scores
    if verbose:
        print('censored score: %s'%censored_score)
        print('uncensored score: %s'%uncensored_score)
   
    #if spam score is higher, mark this as spam
    return (censored_score >= uncensored_score)

In [23]:
predict_text('I think turkey is the worst'.split(), verbose=True)

    word  censored_prob  uncensored_prob     ratio
0      I       0.004329         0.010687  0.405038
1  think       0.000435         0.000534  0.814869
2     is       0.012243         0.010259  1.193344
3    the       0.029813         0.023796  1.252872
4  worst       0.000102         0.000142  0.719002
censored score: -30.9947773282574
uncensored score: -29.92256484105663


False

In [24]:
predictions = x_test.apply(lambda t: predict_text(t.split()))

In [25]:
confusion_matrix(y_test, predictions)
print(classification_report(y_test, predictions))
print("Accuracy: ", mnb.score(x_testcv, y_test))

Fraction censored Correctly Detected: 0.4639423076923077


In [26]:
scores = mnb.predict_proba(x_testcv)
fpr, tpr, _ = roc_curve(y_test, scores[:, 1])
plt.plot(fpr, tpr)

#Plot
plt.title('ROC Curves')
plt.xlabel('False positive rate'); plt.ylabel('True positive rate')
plt.legend(['Logistic Regression'])
plt.show()

Fraction uncensored identified as censored: 0.7956730769230769
